In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 데이터 불러오기

In [2]:
# 데이터 경로 지정
# DATA_PATH = "/gdrive/MyDrive/데이터 청년 캠퍼스 - 1조/딥러닝 - 코인 예측 모델/2.data/"   # google drive
DATA_PATH = "data/"
MINUTE30_PATH = "minute30/"
MINUTE60_PATH = "minute60/"
BTC_PATH = "KRW-BTC.csv"
ETH_PATH = "KRW-ETH.csv"
BTC_HIS = "BTC_2012_2021.csv"

In [3]:
# 데이터 불러오기
data_btc30 = pd.read_csv(DATA_PATH + MINUTE30_PATH + BTC_PATH, index_col=0)
data_btc60 = pd.read_csv(DATA_PATH + MINUTE30_PATH + BTC_PATH, index_col=0)
data_eth30 = pd.read_csv(DATA_PATH + MINUTE60_PATH + ETH_PATH, index_col=0)
data_eth60 = pd.read_csv(DATA_PATH + MINUTE60_PATH + ETH_PATH, index_col=0)
data_btc_his =  pd.read_csv(DATA_PATH + BTC_HIS, index_col=0)

ALL_DATA = [data_btc60, data_btc30, data_eth30, data_eth60]

In [4]:
btc_df = data_btc_his.copy()
btc_df.head()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.39
2011-12-31 07:53:00,4.39,4.39,4.39,4.39,0.555046,2.436653,4.39
2011-12-31 07:54:00,4.39,4.39,4.39,4.39,0.654511,2.873305,4.39
2011-12-31 07:55:00,4.39,4.39,4.39,4.39,0.753977,3.309958,4.39
2011-12-31 07:56:00,4.39,4.39,4.39,4.39,0.853442,3.746611,4.39


## Feature Engineer

### 이동평균선 구하기
* 5분 30분 60분 1일 3일 7일 30일 90일 180일

In [5]:
sma_col_name = ['sma5m', 'sma30m', 'sma60m', 'sma1d', 'sma3d', 'sma7d', 'sma30d', 'sma90d', 'sma180d']
sma_mins = [5, 30, 60, 1440, 1440*3, 1440*7, 1440*30, 1440*90, 1440*180]
sma_zip = zip(sma_col_name, sma_mins)

In [6]:
def sma_x(data, min_=5):
    
    return data['Close'].rolling(min_).mean()

In [7]:
for name, m in sma_zip:
    btc_df[name] = sma_x(btc_df, m)
btc_df.head()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,sma5m,sma30m,sma60m,sma1d,sma3d,sma7d,sma30d,sma90d,sma180d
2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:53:00,4.39,4.39,4.39,4.39,0.555046,2.436653,4.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:54:00,4.39,4.39,4.39,4.39,0.654511,2.873305,4.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:55:00,4.39,4.39,4.39,4.39,0.753977,3.309958,4.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:56:00,4.39,4.39,4.39,4.39,0.853442,3.746611,4.39,4.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# 앞부분 연속 데이터가 없어 결측치 발생
# 초기에는 변화가 크지 않으므로 뒤에서부터 채움
btc_df.interpolate(limit_direction="backward", inplace=True)
btc_df.head()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,sma5m,sma30m,sma60m,sma1d,sma3d,sma7d,sma30d,sma90d,sma180d
2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.39,4.39,4.39,4.39,4.518308,4.824806,5.432311,6.20297,5.473687,5.378391
2011-12-31 07:53:00,4.39,4.39,4.39,4.39,0.555046,2.436653,4.39,4.39,4.39,4.39,4.518308,4.824806,5.432311,6.20297,5.473687,5.378391
2011-12-31 07:54:00,4.39,4.39,4.39,4.39,0.654511,2.873305,4.39,4.39,4.39,4.39,4.518308,4.824806,5.432311,6.20297,5.473687,5.378391
2011-12-31 07:55:00,4.39,4.39,4.39,4.39,0.753977,3.309958,4.39,4.39,4.39,4.39,4.518308,4.824806,5.432311,6.20297,5.473687,5.378391
2011-12-31 07:56:00,4.39,4.39,4.39,4.39,0.853442,3.746611,4.39,4.39,4.39,4.39,4.518308,4.824806,5.432311,6.20297,5.473687,5.378391
